手撕：多层 MLP + 激活函数 + 前向传播 + 反向传播 + 梯度下降

In [1]:
import numpy as np

# relu激活函数以及其导数
def relu(x):
    return np.maximum(0,x)
def relu_derivative(x):
    return (x>0).astype(float)
# 损失函数以及其导数
def mse(y_true,y_pre):
    return np.mean((y_true-y_pre)**2)
# 损失函数的导数用于反向传播
def mse_derivative(y_true,y_pre):
    return 2*(y_pre-y_true)/y_true.shape[0]

# 初始化网络参数
def initial_parameters(layer_dims):
    # 设置随机种子，使每次初始化结果都一样，方便调试和复现模型训练
    np.random.seed(42)
    # 储存元素
    parameters={}
    # layer_dims[i-1]: 前一层的节点数
    # layer_dims[i]: 当前层的节点数
    # np.random.randn(...): 生成标准正态分布的随机值
    # * 0.1: 缩小初始权重值，避免初始梯度过大（有助于稳定训练）
    for i in range (1,len(layer_dims)):
        parameters['W'+str(i)]=np.random.randn(layer_dims[i-1],layer_dims[i])*0.1
        # 偏置初始化为零，形状为 (1, 当前层神经元数)，即每个神经元一个偏置。
        parameters['b'+str(i)]=np.zeros((1,layer_dims[i]))
    return parameters

# 前向传播
def forward(x,parameters):
    # parameters: 存储所有层参数的字典（包含权重矩阵 W1, W2, ..., WL 和偏置向量 b1, b2, ..., bL）
    cache={'A0':x}
    # 用 cache 字典保存每一层的中间计算结果：包括激活值 A 和线性变换结果 Z
    # 每一层都有一个 W 和 b，所以总参数数是 2L。这里计算出有多少层（不包括输入层）。
    L=len(parameters)//2

    for i in range (1,L+1):
        # 因为要到 L 层，所以得 L+1
        # 线性变换
        Z=np.dot(cache['A'+str(i-1)],parameters['W'+str(i)])+parameters['b'+str(i)]
        if i<L:
            # 隐藏层使用 ReLU 激活函数
            A=relu(Z)
        else:
            # 输出层不使用激活函数（回归）
            A=Z
        cache['A'+str(i)]=A
        cache['Z'+str(i)]=Z
    return cache['A'+str(i)],cache

# 反向传播
def backward_propagation(y_true, parameters, cache):
    # grads: 储存每一层的梯度，如 dW1, db1, …
    grads={}
    # L: 总层数（每层有 W 和 b，所以除以 2）
    L=len(parameters)//2
    # 求输出层的导数，也就是 mse 的导数
    # 损失函数的梯度（对输出层激活 A 的导数）
    # cache['A'+str(L)]是最后一层的输出
    dA=mse_derivative(y_true,cache['A'+str(L)])
    for i in reversed(range(1,L+1)):
        dZ=dA
        if i<L:
            # 对隐藏层：激活函数是 ReLU
            dZ=dA*relu_derivative(cache['Z'+str(i)])
        dW=np.dot(cache['A'+str(i-1)].T,dZ)
        #  keepdims=True：保留二维形状 (1, n_l)
        db=np.sum(dZ, axis=0, keepdims=True)
        dA=np.dot(dZ,parameters['W'+str(i)].T)
        grads['dW'+str(i)]=dW
        grads['db'+str(i)]=db
    
    return grads

# 参数更新
def update_parameters(parameters, grads, learning_rate):
    L=len(parameters)//2
    for i in range (1,L+1):
        # 跟新参数
        parameters['W'+str(i)]=parameters['W'+str(i)]-learning_rate*grads['dW'+str(i)]
        parameters['b'+str(i)]=parameters['b'+str(i)]-learning_rate*grads['db'+str(i)]
    return parameters

# 训练模型
def train_model(X, y, hidden_layers=[10, 20, 10], learning_rate=0.01, epochs=1000):
    # 得到参数的维度
    n_features=X.shape[1]
    # 增加输入层+隐藏层+输出层
    layer_dims=[n_features]+hidden_layers+[1]
    # 初始化参数
    parameters=initial_parameters(layer_dims)
    for i in range(epochs):
        # 得到输出值和中间参数的储存
        y_pre,cache=forward(X,parameters)
        # 计算损失
        loss=mse(y,y_pre)
        # 反向传播计算梯度
        grad=backward_propagation(y,parameters,cache)
        # 更新参数
        parameters=update_parameters(parameters,grad,learning_rate)
        # 打印
        if i %100 ==0 or i == epochs-1:
            print(f'Epoch:{i},Loss:{loss}')
    return parameters

# 示例数据
np.random.seed(0)
X = np.random.randn(100, 3)
true_weights = np.array([[2.0], [-3.0], [1.5]])
y = X @ true_weights + 0.5 * np.random.randn(100, 1)

# 训练 MLP
trained_params = train_model(X, y, hidden_layers=[16, 8], learning_rate=0.01, epochs=1000)

Epoch:0,Loss:17.268411061762947
Epoch:100,Loss:0.9615527036924175
Epoch:200,Loss:0.360713848580816
Epoch:300,Loss:0.23310955439227385
Epoch:400,Loss:0.18711196944197891
Epoch:500,Loss:0.17493387668927263
Epoch:600,Loss:0.1690702206898368
Epoch:700,Loss:0.1649386811125157
Epoch:800,Loss:0.16256309176104078
Epoch:900,Loss:0.1606459903567892
Epoch:999,Loss:0.15882770556443898
